In [2]:
import os
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
DATA_ROOT = Path(r"C:\Rashed\8119\Mid Term\scripts\data\raw")
SAVE_DIR = Path("results/classifier_A")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4
NUM_CLASSES = 2

In [5]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.9, 1.0)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_ds = datasets.ImageFolder(DATA_ROOT / "train", transform=train_transform)
test_ds  = datasets.ImageFolder(DATA_ROOT / "test", transform=test_transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Loaded dataset → {len(train_ds)} train, {len(test_ds)} test images, classes: {train_ds.classes}")


Loaded dataset → 20000 train, 5000 test images, classes: ['cat', 'dog']


In [ ]:
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
for param in model.features.parameters():
    param.requires_grad = False  

# replace classifier head
in_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, NUM_CLASSES)
)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\rhasan7/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 58.6MB/s]


In [9]:
from sklearn.metrics import roc_auc_score


In [10]:
for epoch in range(EPOCHS):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {total_loss/len(train_loader):.4f}  Acc: {acc:.2f}%")


model.eval()
y_true, y_pred, y_prob = [], [], []
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(DEVICE)
        outputs = model(imgs)
        probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())
        y_prob.extend(probs)

# Metrics
report = classification_report(y_true, y_pred, target_names=train_ds.classes, digits=3)
auc = roc_auc_score(y_true, y_prob)

print("\nClassification Report:\n", report)


Epoch [1/10] Loss: 0.2626  Acc: 88.15%
Epoch [2/10] Loss: 0.2575  Acc: 88.35%
Epoch [3/10] Loss: 0.2555  Acc: 88.76%
Epoch [4/10] Loss: 0.2543  Acc: 88.70%
Epoch [5/10] Loss: 0.2566  Acc: 88.59%
Epoch [6/10] Loss: 0.2546  Acc: 88.75%
Epoch [7/10] Loss: 0.2504  Acc: 88.72%
Epoch [8/10] Loss: 0.2487  Acc: 88.99%
Epoch [9/10] Loss: 0.2460  Acc: 89.19%
Epoch [10/10] Loss: 0.2374  Acc: 90.06%

Classification Report:
               precision    recall  f1-score   support

         cat      0.937     0.940     0.938      2500
         dog      0.940     0.936     0.938      2500

    accuracy                          0.938      5000
   macro avg      0.938     0.938     0.938      5000
weighted avg      0.938     0.938     0.938      5000



In [12]:
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=train_ds.classes, yticklabels=train_ds.classes)
plt.title("Model A (Original HR images)")
plt.xlabel("Predicted"); plt.ylabel("True")
plt.tight_layout()
plt.savefig(SAVE_DIR / "confusion_matrix_A.png")
plt.close()


os.makedirs("results/classifier_A", exist_ok=True)
torch.save(model.state_dict(), "results/classifier_A/classifier_A.pth")
with open("results/classifier_A/report_A.txt", "w") as f:
    f.write(report + f"\nAUC: {auc:.3f}\n")
